In [76]:
import pandas as pd
import numpy as np
from math import ceil

In [77]:
teams = pd.read_csv("2021_teams.csv")
games = pd.read_csv("./games.csv")

In [78]:
numTeams = len(teams)
numGames = len(games)

In [79]:
# Create the Colley matrix

colley_matrix = 2*np.diag(np.ones(numTeams))
b = np.ones(numTeams)
k = 0


In [80]:
day_before_season = games.loc[0,'days_since_timestart'] - 1
last_day_of_season = games.loc[len(games)-1,'days_since_timestart']

In [81]:
weight_home_win = 1
weight_away_win = 1
weight_neutral_win = 1
segment_weights = [1,1,2,3,5]
num_segments = len(segment_weights)

In [82]:
for i in range(numGames):
  [current_day,date,team_1_id, team_1_homefield, team_1_score, team_2_id, team_2_homefield, team_2_score] = games.loc[i,:].values

  team_1_index = team_1_id - 1
  team_2_index = team_2_id - 1

  weight_index = ceil(num_segments*((current_day-day_before_season)/(last_day_of_season-day_before_season)))-1
  time_weight = segment_weights[weight_index]

  # If Team 1 won the game
  if team_1_score > team_2_score:
    # If team 1 was the home team
    if team_1_homefield == 1:
      gameWeight = weight_home_win * time_weight
    # If team 1 was the away team
    elif team_1_homefield == -1:
      gameWeight = weight_away_win * time_weight
    # If team 1 was a neutral team
    else:
      gameWeight = weight_neutral_win * time_weight
  else:
    if team_2_homefield == 1:
      gameWeight = weight_home_win * time_weight
    elif team_2_homefield == -1:
      gameWeight = weight_away_win * time_weight
    else:
      gameWeight = weight_neutral_win * time_weight

  # Update the Colley Matrix and RHS
  colley_matrix[team_1_index,team_2_index] -= gameWeight
  colley_matrix[team_2_index,team_1_index] -= gameWeight

  colley_matrix[team_1_index,team_1_index] += gameWeight
  colley_matrix[team_2_index,team_2_index] += gameWeight

  if team_1_score > team_2_score:
    b[team_1_index] += gameWeight/2
    b[team_2_index] -= gameWeight/2
  else:
    b[team_1_index] -= gameWeight/2
    b[team_2_index] += gameWeight/2

In [83]:
# Calculate the linear system
r = np.linalg.solve(colley_matrix,b)
iSort = np.argsort(-r)

In [85]:
print('\n\n************** COLLEY Rating Method **************\n')
print('===========================')
print('Rank   Rating    Team   ')
print('===========================')
if k==0:
    numberTeamToPrint = numTeams
else:
    numberTeamToPrint = k

colley_ratings = pd.DataFrame(columns=['rank','rating','team'])

for i in range(numberTeamToPrint):
    print(f'{i+1:4d}   {r[iSort[i]]:.5f}  {teams.loc[iSort[i],"team"]}')
    colley_ratings.loc[i,:] = [i+1,r[iSort[i]],teams.loc[iSort[i],"team"]]

print('')   # extra carriage return



************** COLLEY Rating Method **************

Rank   Rating    Team   
   1   1.02725  Arizona
   2   1.02443  Gonzaga
   3   1.02317  Tennessee
   4   0.99636  Kentucky
   5   0.99634  Kansas
   6   0.98064  Villanova
   7   0.96591  Arkansas
   8   0.95580  Boise_St
   9   0.94698  Purdue
  10   0.93433  Auburn
  11   0.93049  Houston
  12   0.92648  St_Mary's_CA
  13   0.92605  Duke
  14   0.92299  Providence
  15   0.91988  Colorado_St
  16   0.89981  UCLA
  17   0.88598  Baylor
  18   0.88020  Texas_Tech
  19   0.86790  San_Diego_St
  20   0.86756  Connecticut
  21   0.86390  Davidson
  22   0.86369  Wisconsin
  23   0.85862  Iowa
  24   0.85365  Murray_St
  25   0.84994  Illinois
  26   0.84705  Memphis
  27   0.83585  North_Carolina
  28   0.83484  SMU
  29   0.82912  Wyoming
  30   0.82357  North_Texas
  31   0.82057  USC
  32   0.81399  S_Dakota_St
  33   0.81285  VCU
  34   0.81120  New_Mexico_St
  35   0.80489  Virginia_Tech
  36   0.80369  Creighton
  37   0.79147  

In [86]:
colley_ratings.to_csv("colley_ratings.csv",index=False)